<div align="center">
    <h1><strong>Transferencia de Calor 2D - Smith Hutton</strong></h1>
    <h2>Juan David Argüello Plata</h2>
    <img src="Images/uis.png" style="width: 300px;" />
</div>

## __1. Planteamiento del Problema__

<div align="justify">
    Se busca analizar, mediante métodos <i>analíticos</i> y <i>numéricos</i>, la transferencia de calor de la barra mostrada en la Figura 1.
</div>

<br>

<div align="center">
    <img src="Images/Esquema_3D/Esquema.PNG" style="width: 500px;" />
</div>

<div align="center">
    <i>Figura 1.</i> Geometría del problema.
</div>

<div align="justify">
    Se toman las siguientes suposiciones:
</div>

* Conductividad térmica constante.
* Conducción 2D.
* Sin generación.
* Campo de velocidades conocido.
* Transferencia de calor transitoria.

<div align="justify">
    Los datos del problema se pueden especificar a continuación.
</div>

In [1]:
data = {
    'Geometría': {
        'W': {
            'Valor':2,
            'Units': 'm'
        },
        'H': {
            'Valor': 1,
            'Units': 'm'
        }
    },
    'Propiedades': {
        'T_0': {
            'Valor': 5,
            'Units': '°C'
        },
        'T_{infty}': {
            'Valor': 0,
            'Units': '°C'
        },
        'K': {
            'Valor': 55,
            'Units': 'W/m \, °C'
        },
        'h': {
            'Valor': 10,
            'Units': 'W/m^2 \, °C'
        },
        'Q_g': {
            'Valor': 0,
            'Units': 'W/m^3'
        }
    }
}

from IPython.display import HTML, display
for c in data:
    for d in data[c]: 
        text = '$' + d + '= ' + str(data[c][d]['Valor']) + '' + data[c][d]['Units'] + '$'
        display(HTML(text))

## __2. Solución Analítica__

<div align="justify">
    Se desarrolla el análisis sobre lo mostrado en la Figura 2.
</div>

<div align="center">
    <img src="Images/Esquema/Esquema.png" style="width: 600px;" />
</div>

<div align="center">
    <i>Figura 2.</i> Problema de estudio con condiciones de frontera.
</div>

<div align="justify">
    De la Figura 2, $\theta$ es la temperatura adimensional y $Bi$ es el número de Biot.
    $$
    \begin{equation}
        \theta (x^*,y^*) \equiv \frac{T(x,y) - T_{\infty}}{T_0 - T_{\infty}}
        \tag{1}
        \label{teta}
    \end{equation}
    $$
    El número de Biot se puede apreciar en la Ecuación \ref{Biot}.
    $$
    \begin{equation}
        Bi = \frac{h W}{K}
        \tag{2}
        \label{Biot}
    \end{equation}
    $$
    El análisis se desarrollará con variables adimensionales. 
    $$
    \begin{equation}
        H ^* \equiv  \frac{H}{W} \\
        x ^* \equiv \frac{x}{W} \\
        y ^* \equiv \frac{y}{H}
    \end{equation}
    $$
</div>

In [2]:
from sympy import init_session
init_session(use_latex=True)

IPython console for SymPy 1.4 (Python 3.7.4-32-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.4/



In [ ]:
#Constantes adimensionales
H = data['Geometría']['H']['Valor']/data['Geometría']['W']['Valor']
Bi = data['Geometría']['W']['Valor']*\
data['Propiedades']['h']['Valor']/data['Propiedades']['K']['Valor']
theta_0 = (data['Propiedades']['T_0']['Valor'] - \
           data['Propiedades']['T_{infty}']['Valor'])/\
(data['Propiedades']['T_0']['Valor'] - \
           data['Propiedades']['T_{infty}']['Valor'])

#Planteamiento de la ecuación diferencial
#thetaX, thetaY = map(Function, 'XY')
theta = Function('\\theta')
Ec = Eq(theta(x,y).diff(x,2) + theta(x,y).diff(y,2),0)
Ec

In [ ]:
#Condiciones de frontera
bc = {
    theta(x,0):theta_0,
    theta(x,y).diff(x).subs(x,0): 0,
    theta(x,y).diff(y).subs(y,1): 0,
    theta(x,y).diff(x).subs(x,1): -Bi*theta(x,y).subs(x,1)
}
"""
sol = pdsolve(Ec, theta(x,y))
sol
"""
bc

## __3. Solución Numérica__

<div align="justify">
    La geomtería dada es la siguiente:
</div>

In [ ]:
%matplotlib inline
from App.Preprocessing.Geometry import *
Geo(data['Geometría'], True)

In [6]:
import math
x =0
1+math.tanh(10*(2*x+1))

### 3.1. Mallado

<div align="justify">
    Se inicia realizando la discretización del dominio.
</div>

In [3]:
%matplotlib inline
from App.Preprocessing.Mesh import Malla
import sqlite3 as sql
from ipywidgets import *
def Mesh(xsub,ysub, numN = False):
    Malla((data['Geometría']['W']['Valor']/xsub,
           data['Geometría']['H']['Valor']/ysub),
           (data['Propiedades']['T_0']['Valor'],
            data['Propiedades']['T_{infty}']['Valor']),
           False,
           (numN, False),
           data['Geometría'])
    #Conexión con base de datos
    con = sql.connect('App/data.db')
    text = ["SELECT * FROM ", " ORDER BY ", " DESC LIMIT 1"]
    n =  con.execute(text[0]+' nodes '+text[1]+' NodeID '+text[2]).fetchall()
    el = con.execute(text[0]+' elements '+text[1]+' ElID '+text[2]).fetchall()
    con.close()
    display(HTML("Número de nodos: " + str(n[0][0])))
    display(HTML("Número de elementos: " + str(el[0][0])))
    return data['Geometría']['W']['Valor']/xsub, data['Geometría']['H']['Valor']/ysub
ElData = interactive(Mesh,
                     xsub=(3,6,1),
                     ysub=(3,6,1),
                     numN = False, 
                     continous_update=False)
display(ElData)

interactive(children=(IntSlider(value=4, description='xsub', max=6, min=3), IntSlider(value=4, description='ys…

### 3.2. Planteamiento y solución del sistema matricial

<div align="justify">
    El planteamiento matricial se desarrolla con base en las siguietes relaciones matemáticas:
    $$
    \begin{equation}
        a_p T_p = a_E T_E + a_W T_W + a_N T_N + a_S T_S + b
        \tag{3}
        \label{Ts}
    \end{equation}
    $$
    $$
    \begin{equation}
        u = 2y \left(1-x^2 \right) \\
        v = -2x \left(1-y^2 \right)
    \end{equation}
    $$
    Dónde $b$ es la generación de calor y: 
    $$
    \begin{equation}
        a_p = a_E + a_W + a_N + a_S + a_{p0} + \left(F_E-F_W+F_N-F_S \right) \\
        F_E = F_W = \rho u \Delta y \\
        F_N = F_S = \rho v \Delta x \\
        a_{p0} = \rho \frac{\Delta x \Delta y}{\Delta t} \\
        b = a_{p0} T _p ^0 + Q_g \Delta x \Delta y \\
        a_E = D_E A_{pE} + max \left(-F_E,0 \right) \\
        a_W = D_W A_{pW} + max \left(F_W,0 \right) \\
        a_N = D_N A_{pN} + max \left(-F_N,0 \right) \\
        a_S = D_S A_{pS} + max \left(F_S,0 \right) \\
        A_{pi} = max \left(0 \, , 1-0.5 \mid P_i \mid \right) \\
        P_i = \frac{F_i}{D_i} = \frac{\rho u A}{K}
    \end{equation}
    $$
    Para la malla mostrada anteriormente, el sistema de ecuaciones general es el siguiente:
</div>

In [8]:
#Lectura de la base de datos
con = sql.connect('App/data.db')
nodos = con.execute("SELECT * FROM nodes ORDER BY ID ASC").fetchall()
elementos = con.execute("SELECT * FROM elements ORDER BY p ASC").fetchall()
con.close()
#Símbolos
Ts = symbols("T_1:" + str(len(nodos)+1))
a_p, a_E, a_W, a_S, a_N, Q_g = symbols("a_p, a_E, a_W, a_S, a_N, Q_g")
Dx, Dy, dx, dy = Symbol("\\Delta  x"), Symbol("\\Delta  y"), Symbol("\\delta  x"), Symbol("\\delta  y")
#Planteamiento de ecuaciones y suposiciones
Ecuaciones = []
sol = {}
suma = 0
for i in range(len(nodos)):
    if nodos[i][-1] is None:
        Ec = Eq(a_p*Ts[elementos[suma][3]-1],a_E*Ts[elementos[suma][4]-1]+\
                a_W*Ts[elementos[suma][5]-1]+a_N*Ts[elementos[suma][1]-1]+\
                a_S*Ts[elementos[suma][2]-1]+Q_g*Dx*Dy)
        sol[Ts[elementos[suma][3]-1]] = solve(Ec, Ts[elementos[suma][3]-1])[0]
        suma += 1
    else:
        Ec = Eq(Ts[i], nodos[i][-1])
    Ecuaciones.append(Ec)
sup = {}
for el in elementos:
    sup[Ts[el[3]-1]] = 0
Ecuaciones

In [9]:
elementos

[(1, 6, 4, 5, 10, 1, None, None),
 (2, 7, 5, 6, 11, 2, None, None),
 (3, 8, 6, 7, 12, 3, None, 0.0),
 (4, 11, 9, 10, 15, 5, None, None),
 (5, 12, 10, 11, 16, 6, None, None),
 (6, 13, 11, 12, 17, 7, None, 0.0),
 (7, 16, 14, 15, 19, 10, 0.0, None),
 (8, 17, 15, 16, 20, 11, 0.0, None),
 (9, 18, 16, 17, 21, 12, 0.0, 0.0)]

#### 3.2.1. Solución mediante _Gauss - Seidel_

<div align="justify">
    <i> Gauss - Seidel</i> plantea una metodología de solución y comprobación iterativa. Se parte de la solución "trivial" como punto de partida.
</div>

In [ ]:
for s in sol:
    sol[s] = sol[s].subs(sup)
sol

In [ ]:
sol

In [ ]:
sup

In [ ]:
#Inicialización de la matriz
M = []
for i in range(len(nodos)):
    vec = []
    for j in range(len(nodos)):
        vec.append(0)
    M.append(vec)
for i in range(len(nodos)):
    M[i][i] = solve(Ecuaciones[i], Ts[i])[0]
SM = Matrix(M)
SM

Con la matriz simbólica podemos calcular los eigenvalores y eigenvectores.

In [ ]:
#SM.eigenvals()

In [ ]:
ElData.result

In [ ]:
ec = Eq(2*Ts[0] + Ts[1], 5*Ts[4])
sol = solve(ec, Ts[1])
sol